In [38]:
from cognite.client.data_classes import FileMetadataWrite
from cognite.client.data_classes.data_modeling import SpaceApply
from cognite.client.data_classes.data_modeling.cdm.v1 import CogniteFileApply
from cognite.client.exceptions import CogniteException
from IPython.display import display

from tests.auth_utils import get_toolkit_client

In [39]:
client = get_toolkit_client("dev-cluster.env", enable_set_pending_ids=True)

Found dev-cluster.env file in repository root. Loaded variables from dev-cluster.env file.


In [40]:
SPACE = "sp_upgrade"

In [41]:
client.data_modeling.spaces.apply(SpaceApply(space=SPACE))

,value
space,sp_upgrade
is_global,False
last_updated_time,2025-07-04 09:38:40.297000
created_time,2025-07-04 09:38:40.297000


# Cascading Delete

In [42]:
file = FileMetadataWrite(
    external_id="ts_case2_cascading_delete",
    name="Case 2 Cascading Delete",
)

In [43]:
# Ensure the file does not exist before running the test
client.files.delete(external_id=file.external_id, ignore_unknown_ids=True)

In [44]:
created, _ = client.files.create(file)
created

,value
external_id,ts_case2_cascading_delete
name,Case 2 Cascading Delete
labels,[]
id,7022177159174571
created_time,2025-07-10 05:37:26.858000
last_updated_time,2025-07-10 05:37:26.858000
uploaded,False


In [45]:
filecontent = b"This is filecontent"

In [46]:
client.files.upload_content_bytes(filecontent, external_id=created.external_id)

,value
external_id,ts_case2_cascading_delete
name,Case 2 Cascading Delete
id,7022177159174571
created_time,2025-07-10 05:37:26.858000
last_updated_time,2025-07-10 05:37:26.858000
uploaded,False


In [47]:
try:
    updated = client.files.set_pending_ids((SPACE, file.external_id), external_id=file.external_id)
except CogniteException as e:
    print(f"\033[91mError\033[0m {e.code} {e.message!s}")
else:
    display(updated)

,value
external_id,ts_case2_cascading_delete
name,Case 2 Cascading Delete
id,7022177159174571
created_time,2025-07-10 05:37:26.858000
last_updated_time,2025-07-10 05:37:30.356000
uploaded,True
uploaded_time,2025-07-10 05:37:29.833000
pending_instance_id,"{'space': 'sp_upgrade', 'external_id': 'ts_cas..."


In [48]:
cognite_file = CogniteFileApply(
    space=SPACE,
    external_id=file.external_id,
    name=file.name,
)

In [49]:
created_dm = client.data_modeling.instances.apply(cognite_file).nodes
created_dm

,space,instance_type,external_id,version,was_modified,last_updated_time,created_time
0,sp_upgrade,node,ts_case2_cascading_delete,1,True,2025-07-10 05:37:31.585,2025-07-10 05:37:31.585


In [50]:
try:
    client.files.delete(external_id=file.external_id)
except CogniteException as e:
    print(f"\033[91mError\033[0m {e.code} {e.message!s}")
else:
    print("\033[92mFile deleted successfully\033[0m")

Error 403 Resource not found. This may also be due to insufficient access rights.


In [51]:
deleted = client.data_modeling.instances.delete(cognite_file.as_id()).nodes
deleted

[NodeId(space='sp_upgrade', external_id='ts_case2_cascading_delete')]

In [54]:
try:
    client.files.retrieve_multiple(external_ids=[file.external_id])
except CogniteException as e:
    print(f"\033[91mError\033[0m {e!s}")
else:
    print("\033[92mFile  still exists\033[0m")

Error Not found: [{'externalId': 'ts_case2_cascading_delete'}]
The API Failed to process some items.
Successful (2xx): []
Unknown (5xx): []
Failed (4xx): [{'externalId': 'ts_case2_cascading_delete'}, ...]
